In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from testMail import Mail

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras.losses import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from tensorflow.keras import Model

In [2]:
print(tf.__version__)
print(tf.test.gpu_device_name())
# tf.debugging.set_log_device_placement(True)

2.2.0-rc3
/device:GPU:0


# 경로 설정

In [3]:
filename = 'CNN_1'

base = os.getcwd()
data_path = os.path.join(base, 'data')
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')
submit_path = os.path.join(base, 'submit')
model_path = os.path.join(base, 'model', filename)
callback_path = os.path.join(base, 'callback', filename)

if not os.path.isdir(model_path):
    os.mkdir(model_path)

if not os.path.isdir(callback_path):
    os.mkdir(callback_path)

def load(name):
    return np.load(os.path.join(data_path, "{}.npy".format(name)))

# 데이터 로드

In [5]:
data = load('dl_train')

# metrics 설정

In [6]:
np.random.seed(7)
tf.random.set_seed(7)
# tf.random.set_random_seed(7)

In [7]:
def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

def score(y_val, pred):
    f_value = fscore(y_val, pred)
    mae_value = maeOverFscore(y_val, pred)
    
    print("fscore : {} \t maeOverFscore : {}".format(f_value, mae_value))
    
    return (f_value, mae_value)

## 학습 시각화

In [8]:
def visual_train(history):
    hist = history.history
    fig, ax = plt.subplots(1,3, figsize=(24,8))

    for i, key in enumerate(hist.keys()):
        ax[i].set_title(key)
        ax[i].plot(range(1, len(hist[key]) + 1), hist[key], 'b')
        ax[i].set_xlabel('epoch')
        ax[i].set_ylabel(key.split('_')[0])
#         ax[i].legend(loc='upper left')

# 콜백 설정

In [9]:
def create_checkpoint_weights(name):
    checkpoint_path = os.path.join(callback_path, "{}.ckpt".format(name))
    
    return ModelCheckpoint(checkpoint_path,
                           save_weights_only=True,
                           verbose=1)

# 모델 설계

In [10]:
def build_model(dropout=0.25, l1_value=0.01):
    input_layer = Input((40, 40, 14))

    # 40 x 40 -> 20 x 20
    out1_1 = Conv2D(32, (1, 1), activation="relu", padding="same")(input_layer)
    out1_2 = Conv2D(32, (2, 2), activation="relu", padding="same")(input_layer)
    out1_3 = Conv2D(32, (3, 3), activation="relu", padding="same")(input_layer)

    out1 = concatenate([out1_1, out1_2, out1_3])
    out1 = BatchNormalization()(out1)
    out1 = MaxPooling2D(2, 2)(out1)
    out1 = Dropout(dropout)(out1)
#     print("out1 :", out1.shape)


    # 20 * 20 -> 10 * 10
    out2_1 = Conv2D(32, (1, 1), activation="relu", padding="same")(out1)
    out2_2 = Conv2D(64, (2, 2), activation="relu", padding="same")(out1)
    out2_3 = Conv2D(64, (3, 3), activation="relu", padding="same")(out1)
    
    out2 = concatenate([out2_1, out2_2, out2_3])
    out2 = BatchNormalization()(out2)
    out2 = Dropout(dropout)(out2)

    out2 = Conv2D(96, (3,3), padding="same", activation='relu', kernel_regularizer=l1(l1_value))(out2)
    out2 = BatchNormalization()(out2)
    
    out2 = concatenate([out2, out2_1, out2_2, out2_3])
    out2 = MaxPooling2D(2, 2)(out2)
    out2 = Dropout(dropout)(out2)
#     print("out2 :", out2.shape)
    
    
    # 10 * 10
    conv1 = Conv2D(16, (1, 1), activation="relu", padding="same")(out2)
    conv2 = Conv2D(32, (2, 2), activation="relu", padding="same")(out2)
    conv3 = Conv2D(32, (3, 3), activation="relu", padding="same")(out2)

    conv = concatenate([conv1, conv2, conv3])
    conv = BatchNormalization()(conv)
    conv = Dropout(dropout)(conv)
    
    conv = Conv2D(48, (3,3), padding="same", activation='relu', kernel_regularizer=l1(l1_value))(conv)
    conv = BatchNormalization()(conv)
    
    conv = concatenate([conv, conv1, conv2, conv3])
    conv = Dropout(dropout)(conv)
#     print("conv :", conv.shape)

    
    # 10 * 10 -> 20 * 20
    out3_1 = Conv2DTranspose(32, (1,1), strides=(2, 2), padding='same')(conv)
    out3_2 = Conv2DTranspose(64, (2,2), strides=(2, 2), padding='same')(conv)
    out3_3 = Conv2DTranspose(64, (3,3), strides=(2, 2), padding='same')(conv)
    
    out3 = concatenate([out3_1, out3_2, out3_3, out2_1, out2_2, out2_3])
    out3 = BatchNormalization()(out3)
    out3 = Dropout(dropout)(out3)
    
    out3 = Conv2D(64, (3,3), padding="same", activation='relu', kernel_regularizer=l1(l1_value))(out3)
    out3 = BatchNormalization()(out3)
    
    out3 = concatenate([out3, out3_1, out3_2, out3_3, out2_1])
    out3 = Dropout(dropout)(out3)
#     print("out3 :", out3.shape)
    
    
    # 20 * 20 -> 40 * 40
    out4_1 = Conv2DTranspose(16, (1,1), strides=(2, 2), padding='same')(out3)
    out4_2 = Conv2DTranspose(32, (2,2), strides=(2, 2), padding='same')(out3)
    out4_3 = Conv2DTranspose(32, (3,3), strides=(2, 2), padding='same')(out3)

    out4 = concatenate([out4_1, out4_2, out4_3, out1_1, out1_2, out1_3])
    out4 = BatchNormalization()(out4)
    out4 = Dropout(dropout)(out4)
    
    out4 = Conv2D(32, (3,3), padding="same", activation='relu', kernel_regularizer=l1(l1_value))(out4)
    out4 = BatchNormalization()(out4)

    out4 = concatenate([out4, out4_1, out4_2, out4_3, out1_1])
    out4 = Dropout(dropout)(out4)
#     print("out4 :", out4.shape)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu', kernel_regularizer=l1(l1_value))(out4)
#     print("output : ", output_layer.shape)
    
    return Model(input_layer, output_layer)

# 하이퍼 파라미터

In [11]:
epochs=10
learning_rate=1e-4
batch_size=512
shuffle=True

# 모델 생성

In [12]:
# with tf.device(tf.test.gpu_device_name()):
model = build_model()
model.compile(loss=mean_absolute_error, optimizer=Adam(learning_rate), metrics=[maeOverFscore_keras, fscore_keras])

# 모델 학습

In [ ]:
historys = list()

with tf.device(tf.test.gpu_device_name()):
    kfold = KFold(n_splits=4, random_state=7, shuffle=True)
    
    for i, (train_idx, val_idx) in enumerate(kfold.split(data)):
        print("{} : train".format(i))
        x_train = data[train_idx, :, :, :-1]
        y_train = data[train_idx, :, :,  -1].reshape(x_train.shape[0], 40, 40, 1)
        
        x_val = data[val_idx, :, :, :-1]
        y_val = data[val_idx, :, :,  -1].reshape(x_val.shape[0], 40, 40, 1)
        
        cp_callback = create_checkpoint_weights("train_{}".format(i))
        
        history = model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), \
                            epochs=epochs, batch_size=batch_size, shuffle=shuffle, \
                            callbacks = [cp_callback], \
                            verbose=1, use_multiprocessing=True)
        
        historys.append(history)
        visual_train(history)

### 학습 시간
- 1 epochs : 2543s (42m)
- 2 epochs : 2549s 

In [ ]:
for history in historys:
    visual_train(history)

## 재 학습

In [ ]:
epochs=6
learning_rate=1e-4

In [ ]:
with tf.device(tf.test.gpu_device_name()):
    model = build_model(start_neurons)
    model.compile(loss=mean_absolute_error, optimizer=Adam(learning_rate), metrics=[maeOverFscore_keras, fscore_keras])
    
    cp_callback = create_checkpoint_weights("test")
    
    history = model.fit(x=data[:, :, :, :-1], y=data[:, :, :, -1].reshape(data.shape[0], 40, 40, 1), \
                        epochs=epochs, batch_size=batch_size, shuffle=shuffle, \
                        callbacks = [cp_callback], \
                        verbose=1, use_multiprocessing=True)
    
visual_train(history)

# submit 생성

In [ ]:
def submit(model, name):
    test_data = load('test')
    pred = model.predict(test_data)

    submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
    submission.iloc[:, 1:] = pred.reshape(-1, 1600)

    submission.to_csv(os.path.join(submit_path, "{}.csv".format(name)), index=False)

In [ ]:
submit(model, filename)
model.save(os.path.join(model_path, "{}.h5".format(filename)))

# 메일 보내기

In [ ]:
mail = Mail("rhkd865@gmail.com", "drbhcuuccubulmkd")

mail.send("rhkd865@gmail.com", "AIFrenz2", "Finished DL")
mail.send("rhkd865@naver.com", "AIFrenz2", "Finished DL")

mail.close()